<a href="https://colab.research.google.com/github/pesfahanian/MRNet_V1/blob/main/MRNet_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/MRNet-v1.0

/content/drive/MyDrive/MRNet-v1.0


In [2]:
!ls

csv    train_features	     valid	     valid_features_keras
train  train_features_keras  valid_features


In [3]:
import glob
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.io import imread
from skimage.color import gray2rgb

import keras_preprocessing
from keras.layers import Input
from keras.models import load_model
from keras_preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import DenseNet121

In [4]:
IMG_SIZE = 256

In [5]:
input_image = Input((IMG_SIZE, IMG_SIZE, 3))
feature_extractor = DenseNet121(include_top     =   False,
                                weights         =   'imagenet',
                                input_tensor    =   input_image, 
                                input_shape     =   (IMG_SIZE, IMG_SIZE, 3),
                                pooling         =   'avg')

29089792/29084464 [==============================] - 2s 0us/step


In [6]:
feature_extractor.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [7]:
# # Do this later
# def transform(s):
#     s = s.astype("float32") / 255
#     s[:, :, 0] = (s[:, :, 0] - 0.485) / 0.229
#     s[:, :, 1] = (s[:, :, 1] - 0.456) / 0.224
#     s[:, :, 2] = (s[:, :, 2] - 0.406) / 0.225
#     return s

In [8]:
# For all scans, so the following:
#   - For each slice in the scan, do the following:
#       - Read .npy of scan.
#       - Stack to make it 3D.
#       - Transform.
#       - Convert to Image.
#       - Put the image through the model.
#       - Append slice features to scan feature list
#   - Merge all slice features into one.
#   - Append the result to total feature list.  

In [9]:
def extract_features(s):
    img = gray2rgb(s)
    IMAGE = Image.fromarray(img)
    batch = np.expand_dims(IMAGE, axis=0)
    f = feature_extractor.predict(batch, use_multiprocessing=True, workers=4)
    return f[0]

In [10]:
def merge_features(scan):
    features = []
    for s in scan:
        features.append(extract_features(s))
    features = np.mean(np.array(features), axis=0)
    return features

In [11]:
def feature_juicer(path):
    size = len(glob.glob(path))
    print('Size: {}'.format(size))
    F = []
    for i, g in enumerate(glob.glob(path)):
        print(g)
        scan = np.load(g)
        F.append(merge_features(scan))
        print('Case {} - {}% complete'.format(i, int(100*(i+1)/size)))
        # break
    F = np.array(F)
    return F

In [12]:
axial_train = '/content/drive/MyDrive/MRNet-v1.0/train/axial/*.npy'
axial_valid = '/content/drive/MyDrive/MRNet-v1.0/valid/axial/*.npy'

In [13]:
size = len(glob.glob(axial_train))
print('{} axial scans in train.'.format(size))

1130 axial scans in train.


In [14]:
axial_train_features_keras = feature_juicer(axial_train)

Size: 1130
/content/drive/MyDrive/MRNet-v1.0/train/axial/1035.npy
Case 0 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0609.npy
Case 1 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/1098.npy
Case 2 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0899.npy
Case 3 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0848.npy
Case 4 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/1049.npy
Case 5 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0268.npy
Case 6 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0497.npy
Case 7 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/1111.npy
Case 8 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0780.npy
Case 9 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0751.npy
Case 10 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0330.npy
Case 11 - 1% complete
/content/drive/MyDrive/MRNet-v1.0/train/axial/0027.npy
Case 12 - 1% complet

In [15]:
axial_train_features_keras.shape

(1130, 1024)

In [16]:
with open('/content/drive/MyDrive/MRNet-v1.0/train_features_keras/axial_train_features_keras.npy', 'wb') as f:
    np.save(f, axial_train_features_keras)

In [17]:
size = len(glob.glob(axial_valid))
print('{} axial scans in valid.'.format(size))

120 axial scans in valid.


In [18]:
axial_valid_features_keras = feature_juicer(axial_valid)

Size: 120
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1220.npy
Case 0 - 0% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1137.npy
Case 1 - 1% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1227.npy
Case 2 - 2% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1130.npy
Case 3 - 3% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1229.npy
Case 4 - 4% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1142.npy
Case 5 - 5% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1193.npy
Case 6 - 5% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1145.npy
Case 7 - 6% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1194.npy
Case 8 - 7% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1139.npy
Case 9 - 8% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1170.npy
Case 10 - 9% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1177.npy
Case 11 - 10% complete
/content/drive/MyDrive/MRNet-v1.0/valid/axial/1212.npy
Case 12 - 10% comple

In [19]:
axial_valid_features_keras.shape

(120, 1024)

In [20]:
with open('/content/drive/MyDrive/MRNet-v1.0/valid_features_keras/axial_valid_features_keras.npy', 'wb') as f:
    np.save(f, axial_valid_features_keras)